# Common以下のコード補足

In [5]:
import numpy as np

***
# functions

## softmax関数
xが1次元，2次元の場合に分けて，オーバーフロー対策のために各データから最大値を引く

In [2]:
x = np.random.rand(3, 10)
print(x)
print(x.max(axis=1, keepdims=True))
print(x - x.max(axis=1, keepdims=True))

[[0.33965674 0.47727623 0.94697787 0.20657901 0.4952991  0.59108645
  0.59937686 0.14464453 0.47623614 0.39874405]
 [0.33030764 0.20721946 0.81107601 0.35143251 0.22692903 0.04685336
  0.98986583 0.04784092 0.50473521 0.04822969]
 [0.46343588 0.79592932 0.01589677 0.5903422  0.62348082 0.59518771
  0.03112564 0.36355613 0.52098047 0.64284136]]
[[0.94697787]
 [0.98986583]
 [0.79592932]]
[[-0.60732113 -0.46970164  0.         -0.74039886 -0.45167877 -0.35589143
  -0.34760101 -0.80233334 -0.47074173 -0.54823382]
 [-0.65955819 -0.78264637 -0.17878983 -0.63843333 -0.7629368  -0.94301247
   0.         -0.94202492 -0.48513062 -0.94163614]
 [-0.33249343  0.         -0.78003255 -0.20558712 -0.1724485  -0.20074161
  -0.76480368 -0.43237319 -0.27494885 -0.15308796]]


## cross_entropy_error
- 1データ分
$$
L = - \Sigma_k t_k \log y_k
$$

- バッチ
$$
L = - \frac{1}{N} \Sigma_n \Sigma_k t_{nk} \log y_{nk}
$$

- targetラベルがone-hot-vectorであれば，正解ラベルに対応するlog yを計算するだけ．

In [3]:
# 教師データのone-hot-vectorを正解ラベルのインデックスに変換
t = np.array([[0, 0, 1, 0],
              [1, 0, 0, 0],
              [0, 0, 0, 1]])  # target
y = np.random.rand(*t.shape)  # 入力
print(f'y_org: \n {y}')
print(f't_org: \n {t}')

batch_size = y.shape[0]  # バッチ

t = t.argmax(axis=1)  # targetをone-hot-vectorからインデックスに変換

out = -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size
print(f'out: \n {out}')
print(f't_argmax: \n {t}')

y_org: 
 [[0.7606841  0.20212435 0.95315917 0.73135845]
 [0.10888048 0.46024541 0.85491124 0.67857846]
 [0.61053441 0.10939165 0.11165016 0.12646204]]
t_org: 
 [[0 0 1 0]
 [1 0 0 0]
 [0 0 0 1]]
out: 
 1.444429726237475
t_argmax: 
 [2 0 3]


***
# layers

## SigmoidWithLoss
$$
y = \frac{1}{1 + \exp(-x)}
\\ \\
L = - \{ t \log y + (1 - t) \log(1 - y) \}
$$
- CBOWモデルの高速化(多値分類を二値分類に置き換える)で使用．2クラス分類．
- 答えが不正解のときt = 0, 正解のときt = 1が渡される．

In [4]:
x = np.random.rand(5).reshape(5, 1)  # 入力
y = 1 / (1 + np.exp(-x))  # Sigmoid
# 以下の配列と t = 0 or 1 のラベルをcross_entropy_errorに渡せばよい
print(y)
print(np.c_[1 - y, y])

[[0.60021659]
 [0.67915342]
 [0.62374807]
 [0.73072899]
 [0.56236534]]
[[0.39978341 0.60021659]
 [0.32084658 0.67915342]
 [0.37625193 0.62374807]
 [0.26927101 0.73072899]
 [0.43763466 0.56236534]]


## Embedding
- CBOWモデルの高速化．入力層->中間層の全結合層MatMulレイヤの代わりに使う．
- MuｔMulでやったことは単語IDのone-hot-vectorと重みW_inの積和で，これはW_inから単語IDに対応するインデックスの行を抜き出すことに等しい

In [5]:
# forwardメソッド
W = np.random.rand(5, 3)  # 重み
contexts = np.array([[0, 2],
                     [1, 3],
                     [2, 4],
                     [3, 0],
                     [4, 2]])  # window_size=2, n=6のコンテキスト

print(f'W: \n {W}')

# コンテキストの数と同じだけEmbeddingレイヤを使ってコンテキストの列ごとに処理する
for i in range(contexts.shape[1]) :
    idx = contexts[:, i]
    out = W[idx]
    print(f'out_{i}: \n {out}')

W: 
 [[0.9579406  0.91240616 0.53587186]
 [0.29151932 0.92200388 0.79380447]
 [0.94965922 0.55461428 0.65690186]
 [0.9287799  0.78643294 0.93903434]
 [0.99461602 0.71486261 0.92065095]]
out_0: 
 [[0.9579406  0.91240616 0.53587186]
 [0.29151932 0.92200388 0.79380447]
 [0.94965922 0.55461428 0.65690186]
 [0.9287799  0.78643294 0.93903434]
 [0.99461602 0.71486261 0.92065095]]
out_1: 
 [[0.94965922 0.55461428 0.65690186]
 [0.9287799  0.78643294 0.93903434]
 [0.99461602 0.71486261 0.92065095]
 [0.9579406  0.91240616 0.53587186]
 [0.94965922 0.55461428 0.65690186]]


In [6]:
# backwardメソッド
grad = np.ones_like(W)  # 勾配
dW = grad
dW[...] = 0  # 勾配を初期化

dout = np.arange(W.size).reshape(W.shape)
print(f'dW_init: \n {dW}')
print(f'dout: \n {dout}')
print(f'idx: \n {contexts[:, 1]}')

np.add.at(dW, contexts[:, 1], dout)  # dWの指定した行にdoutの各行を足していく
print(f'dW: \n {dW}')

dW_init: 
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
dout: 
 [[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]]
idx: 
 [2 3 4 0 2]
dW: 
 [[ 9. 10. 11.]
 [ 0.  0.  0.]
 [12. 14. 16.]
 [ 3.  4.  5.]
 [ 6.  7.  8.]]


***
# trainer
## Trainer

In [7]:
# データのシャッフル
np.random.permutation(10)

array([6, 8, 7, 4, 0, 1, 5, 2, 3, 9])

In [8]:
x = np.random.randn(1000)
data_size = len(x)
batch_size = 32
max_iters = data_size // batch_size  # 31
max_epoch = 10
eval_interval = 20

In [9]:
# fitメソッド
for epoch in range(max_epoch):
    print(f'--------------epoch: {epoch}--------------')
    print('ここでデータをシャッフル')
    
    for iters in range(max_iters):
        print(f'--------iters: {iters}--------')
        print('ミニバッチを取得')
        print('順伝播で損失計算')
        print('逆伝播で勾配計算')
        print('max_gradを指定していれば勾配クリッピング')
        print('パラメータ更新 (optimizer.update)')
        print('total_lossに損失の値を足し上げる')
        print('loss_countに損失関数の計算回数をカウント')
        
        if (eval_interval is not None) and (iters % eval_interval) == 0:
            print('------eval------')
            print('損失の平均を計算，出力，リスト追加')
            print('total_loss, loss_countを初期化')

--------------epoch: 0--------------
ここでデータをシャッフル
--------iters: 0--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
------eval------
損失の平均を計算，出力，リスト追加
total_loss, loss_countを初期化
--------iters: 1--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 2--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 3--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 4--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 5--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
lo

順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 10--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 11--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 12--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 13--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 14--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 15--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損

パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 8--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 9--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 10--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 11--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 12--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 13--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------i

--------iters: 14--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 15--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 16--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 17--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 18--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 19--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 20--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメー

--------iters: 10--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 11--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 12--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 13--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 14--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 15--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 16--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメー

In [24]:
# remove_duplicate関数
param0 = np.random.rand(3, 2)
param1 = np.random.rand(3, 2)
param2 = param0

grad0 = np.random.rand(*param0.shape)
grad1 = np.random.rand(*param1.shape)
grad2 = np.random.rand(*param2.shape)

params = [param0, param1, param2]
grads = [grad0, grad1, grad2]

print('params:')
for param in params:
    print(param)

print('')
print('grads: ')
for grad in grads:
    print(grad)

params:
[[0.58331604 0.27074317]
 [0.11218083 0.14660672]
 [0.79242935 0.3949079 ]]
[[0.33153473 0.43594233]
 [0.09589007 0.85243157]
 [0.29871991 0.76725271]]
[[0.58331604 0.27074317]
 [0.11218083 0.14660672]
 [0.79242935 0.3949079 ]]

grads: 
[[0.15103878 0.43179069]
 [0.28360044 0.35381496]
 [0.63575541 0.00825461]]
[[0.2332423  0.45842675]
 [0.76298008 0.65864438]
 [0.49756409 0.49982152]]
[[0.86598958 0.42904669]
 [0.20561794 0.43443419]
 [0.86914614 0.06200609]]


In [28]:
def remove_duplicate(params, grads):
    params, grads = params[:], grads[:]  # copy list

    while True:
        find_flg = False
        L = len(params)

        # 二つのループで全ての組み合わせをチェック
        for i in range(0, L - 1):
            for j in range(i + 1, L):
                # 重みを共有する場合
                if params[i] is params[j]:
                    grads[i] += grads[j]  # 勾配の加算
                    find_flg = True
                    params.pop(j)  # 共有する重みを削除
                    grads.pop(j)  # 共有する重みの勾配を削除
                # 転置行列として重みを共有する場合（weight tying）
                elif params[i].ndim == 2 and params[j].ndim == 2 and \
                     params[i].T.shape == params[j].shape and np.all(params[i].T == params[j]):
                    grads[i] += grads[j].T
                    find_flg = True
                    params.pop(j)
                    grads.pop(j)

                if find_flg: break
            if find_flg: break

        if not find_flg: break

    return params, grads

params_removed, grads_removed = remove_duplicate(params, grads)

In [29]:
print('params_removed: ')
for param in params_removed:
    print(param)

print('')
print('grads_removed: ')
for grad in grads_removed:
    print(grad)

params_removed: 
[[0.58331604 0.27074317]
 [0.11218083 0.14660672]
 [0.79242935 0.3949079 ]]
[[0.33153473 0.43594233]
 [0.09589007 0.85243157]
 [0.29871991 0.76725271]]

grads_removed: 
[[1.88301793 1.28988406]
 [0.69483632 1.22268335]
 [2.37404769 0.1322668 ]]
[[0.2332423  0.45842675]
 [0.76298008 0.65864438]
 [0.49756409 0.49982152]]


***
# time_layers.py
- 5章RNN以降に使うレイヤ

In [6]:
vocab_size = 20  # 語彙数
wordvec_size = 5  # Embeddingで単語を分散表現にしたときの次元数．TimeRNNへの入力の次元数になる．
hidden_size = 3  # 状態ベクトルの次元数
V, D, H = vocab_size, wordvec_size, hidden_size

In [7]:
Wx = (np.random.randn(D, H) / np.sqrt(D)).astype('f')  # xを変換する重み
Wh = (np.random.randn(H, H) / np.sqrt(H)).astype('f')  # h_prevを変換する重み

In [9]:
print('Wx')
print(Wx)
print('Wh')
print(Wh)

Wx
[[-0.26670948 -0.4018052  -1.2186108 ]
 [ 0.24342619 -0.11623837 -0.18561976]
 [ 0.05261231 -1.2191893   0.02221492]
 [-0.34332973 -0.5835534  -1.0494016 ]
 [-0.30865684 -0.41788626  0.11010695]]
Wh
[[ 0.01154494 -1.0220374   0.3808439 ]
 [ 0.28054443 -0.6831529   0.26942965]
 [-0.28431925  1.210614   -0.4191217 ]]
